TODO
- Lacmus Drone Dataset (LaDD) https://habr.com/ru/companies/ods/articles/483616/

In [ ]:
# Импорты
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import json
from typing import Optional, Union

from tqdm import tqdm
import polars as pl
from PIL import ImageDraw
import plotly.express as px


# Настройки для графиков
plt.style.use("default")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 10

In [ ]:
# Настройка путей к данным
BASE_PATH = Path("./dataset")  # Измените на ваш путь

# Пути к датасетам
datasets = {
    "train_s1": BASE_PATH / "01_train-s1__DataSet_Human_Rescue",
    "train_s2": BASE_PATH / "02_second_part_DataSet_Human_Rescue",
    "validation_public": BASE_PATH / "03_validation__DataSet_Human_Rescue/public",
    "validation_private": BASE_PATH / "03_validation__DataSet_Human_Rescue/private",
}

# Проверяем доступность данных
print("🔍 Проверка доступности датасетов:")
available_datasets = {}
for name, path in datasets.items():
    if path.exists():
        available_datasets[name] = path
        print(f"✅ {name}: {path}")
    else:
        print(f"❌ {name}: не найден - {path}")

print(f"\n📊 Доступно датасетов: {len(available_datasets)}")

In [ ]:
def extract_dataset_to_parquet(
    datasets_path: Union[str, Path],
    output_path: Union[str, Path],
    sample_size: Optional[int] = None,
    include_image_stats: bool = True,
) -> str:
    """
    Извлекает полную информацию о датасете и сохраняет в parquet файл

    Ищет изображения и аннотации только в следующих папках:
    - 01_train-s1__DataSet_Human_Rescue (сохраняется как train_s1)
    - 02_second_part_DataSet_Human_Rescue (сохраняется как train_s2)
    - 03_validation__DataSet_Human_Rescue (сохраняется как validation)

    Args:
        datasets_path: Путь к папке содержащей папки датасетов (например '/mnt/data/dataset')
        output_path: Путь для сохранения parquet файла
        sample_size: Ограничить количество изображений для анализа (None = все)
        include_image_stats: Включать ли статистику изображений (размер файла и т.д.)

    Returns:
        Путь к созданному parquet файлу
    """

    datasets_path = Path(datasets_path)
    output_path = Path(output_path)

    if not datasets_path.exists():
        raise ValueError(f"Путь к датасетам не существует: {datasets_path}")

    output_path.parent.mkdir(parents=True, exist_ok=True)

    print(f"🔍 Сканирование датасетов в {datasets_path}")

    # Список для хранения всех данных
    all_data = []

    # Поддерживаемые форматы изображений
    image_extensions = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

    # Определенные папки для поиска
    target_datasets = [
        "01_train-s1__DataSet_Human_Rescue",
        "02_second_part_DataSet_Human_Rescue",
        "03_validation__DataSet_Human_Rescue",
    ]

    # Находим изображения только в указанных папках датасетов
    all_images = []
    for dataset_name in target_datasets:
        dataset_path = datasets_path / dataset_name
        if dataset_path.exists():
            print(f"📁 Сканирую {dataset_name}...")
            for ext in image_extensions:
                found_images = list(dataset_path.rglob(f"*{ext}"))
                all_images.extend(found_images)
                if found_images:
                    print(f"   Найдено {len(found_images)} файлов {ext}")
        else:
            print(f"⚠️ Папка {dataset_name} не найдена")

    if sample_size and len(all_images) > sample_size:
        all_images = np.random.choice(all_images, sample_size, replace=False).tolist()

    print(f"📊 Найдено {len(all_images)} изображений для анализа")

    # Создаем словарь для быстрого поиска аннотаций в тех же папках
    print("🔍 Поиск аннотаций...")
    all_labels = []
    for dataset_name in target_datasets:
        dataset_path = datasets_path / dataset_name
        if dataset_path.exists():
            labels_in_dataset = list(dataset_path.rglob("*.txt"))
            all_labels.extend(labels_in_dataset)

    labels_dict = {label.stem: label for label in all_labels}

    print(f"📝 Найдено {len(all_labels)} файлов аннотаций")

    # Обрабатываем каждое изображение
    for img_path in tqdm(all_images, desc="Обработка изображений"):
        try:
            # Базовая информация об изображении
            img_info = {
                "image_path": str(img_path),
                "image_name": img_path.name,
                "image_stem": img_path.stem,
                "dataset_name": _get_dataset_name(img_path, datasets_path),
                "relative_path": str(img_path.relative_to(datasets_path)),
            }

            # Загружаем изображение для получения размеров
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size
                    img_format = img.format
                    img_mode = img.mode

                img_info.update(
                    {
                        "image_width": img_width,
                        "image_height": img_height,
                        "image_format": img_format,
                        "image_mode": img_mode,
                        "image_aspect_ratio": img_width / img_height,
                        "image_total_pixels": img_width * img_height,
                    }
                )

                if include_image_stats:
                    img_info.update(
                        {
                            "image_size_bytes": img_path.stat().st_size,
                            "image_size_mb": img_path.stat().st_size / (1024 * 1024),
                        }
                    )

            except Exception as e:
                print(f"⚠️ Ошибка при загрузке {img_path}: {e}")
                continue

            # Ищем соответствующую аннотацию
            annotation_path = labels_dict.get(img_path.stem)

            if annotation_path and annotation_path.exists():
                img_info["annotation_path"] = str(annotation_path)
                img_info["has_annotation"] = True

                # Парсим аннотации
                try:
                    with open(annotation_path, "r") as f:
                        lines = [line.strip() for line in f.readlines() if line.strip()]

                    img_info["objects_count"] = len(lines)

                    if len(lines) > 0:
                        # Обрабатываем каждый объект
                        for obj_idx, line in enumerate(lines):
                            parts = line.split()
                            if len(parts) >= 5:
                                class_id = int(parts[0])
                                x_center = float(parts[1])
                                y_center = float(parts[2])
                                width_norm = float(parts[3])
                                height_norm = float(parts[4])

                                # Вычисляем координаты в пикселях
                                width_pix = width_norm * img_width
                                height_pix = height_norm * img_height
                                x_center_pix = x_center * img_width
                                y_center_pix = y_center * img_height

                                # Координаты углов
                                x1 = x_center_pix - width_pix / 2
                                y1 = y_center_pix - height_pix / 2
                                x2 = x_center_pix + width_pix / 2
                                y2 = y_center_pix + height_pix / 2

                                # Площадь и статистики
                                area_pix = width_pix * height_pix
                                area_percent = (
                                    area_pix / (img_width * img_height)
                                ) * 100
                                bbox_aspect_ratio = (
                                    width_pix / height_pix if height_pix > 0 else 0
                                )

                                # Создаем запись для каждого объекта
                                obj_data = img_info.copy()
                                obj_data.update(
                                    {
                                        "object_id": obj_idx,
                                        "class_id": class_id,
                                        "x_center_norm": x_center,
                                        "y_center_norm": y_center,
                                        "width_norm": width_norm,
                                        "height_norm": height_norm,
                                        "x_center_pix": x_center_pix,
                                        "y_center_pix": y_center_pix,
                                        "width_pix": width_pix,
                                        "height_pix": height_pix,
                                        "x1": x1,
                                        "y1": y1,
                                        "x2": x2,
                                        "y2": y2,
                                        "bbox_area_pix": area_pix,
                                        "bbox_area_percent": area_percent,
                                        "bbox_aspect_ratio": bbox_aspect_ratio,
                                    }
                                )

                                all_data.append(obj_data)
                    else:
                        # Изображение без объектов
                        img_info["objects_count"] = 0
                        img_info["object_id"] = None
                        all_data.append(img_info)

                except Exception as e:
                    print(f"⚠️ Ошибка при парсинге аннотации {annotation_path}: {e}")
                    img_info["objects_count"] = 0
                    img_info["annotation_error"] = str(e)
                    all_data.append(img_info)
            else:
                # Изображение без аннотации
                img_info["annotation_path"] = None
                img_info["has_annotation"] = False
                img_info["objects_count"] = 0
                img_info["object_id"] = None
                all_data.append(img_info)

        except Exception as e:
            print(f"⚠️ Общая ошибка при обработке {img_path}: {e}")
            continue

    # Создаем DataFrame
    print("📊 Создание DataFrame...")
    df = pd.DataFrame(all_data)

    # Сортируем по пути к изображению и id объекта
    df = df.sort_values(["image_path", "object_id"]).reset_index(drop=True)

    # Сохраняем в parquet
    parquet_path = output_path
    if not str(parquet_path).endswith(".parquet"):
        parquet_path = output_path / "dataset_info.parquet"

    print(f"💾 Сохранение в {parquet_path}")
    df.to_parquet(parquet_path, index=False)

    # Создаем summary
    summary = {
        "total_images": len(df["image_path"].unique()),
        "total_objects": len(df[df["object_id"].notna()]),
        "datasets": list(df["dataset_name"].unique()),
        "image_formats": list(df["image_format"].unique()),
        "has_annotations": int(df["has_annotation"].sum()),
        "images_without_annotations": int((~df["has_annotation"]).sum()),
        "columns": list(df.columns),
        "file_size_mb": parquet_path.stat().st_size / (1024 * 1024),
    }

    # Сохраняем summary
    summary_path = parquet_path.parent / f"{parquet_path.stem}_summary.json"
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)

    print("\n✅ Готово!")
    print(f"📁 Parquet файл: {parquet_path}")
    print(f"📋 Summary: {summary_path}")
    print(
        f"📊 Обработано: {summary['total_images']} изображений, {summary['total_objects']} объектов"
    )
    print(f"💾 Размер файла: {summary['file_size_mb']:.2f} MB")

    return str(parquet_path)


def _get_dataset_name(img_path: Path, base_path: Path) -> str:
    """Определяет название датасета по пути файла"""
    try:
        relative_path = img_path.relative_to(base_path)
        # Берем первую папку в относительном пути
        parts = relative_path.parts
        if len(parts) > 0:
            dataset_folder = parts[0]
            # Сокращаем длинные названия для удобства
            if dataset_folder == "01_train-s1__DataSet_Human_Rescue":
                return "train_s1"
            elif dataset_folder == "02_second_part_DataSet_Human_Rescue":
                return "train_s2"
            elif dataset_folder == "03_validation__DataSet_Human_Rescue":
                return "validation"
            else:
                return dataset_folder
        else:
            return "unknown"
    except:
        return "unknown"


def load_dataset_info(parquet_path: str) -> pd.DataFrame:
    """
    Загружает информацию о датасете из parquet файла

    Args:
        parquet_path: Путь к parquet файлу

    Returns:
        DataFrame с информацией о датасете
    """
    return pd.read_parquet(parquet_path)


def get_dataset_summary(df: pd.DataFrame) -> dict:
    """
    Создает краткую сводку по датасету

    Args:
        df: DataFrame с информацией о датасете

    Returns:
        Словарь со статистикой
    """
    summary = {
        "total_images": len(df["image_path"].unique()),
        "total_objects": len(df[df["object_id"].notna()]),
        "avg_objects_per_image": df.groupby("image_path")["object_id"].count().mean(),
        "datasets": df["dataset_name"].value_counts().to_dict(),
        "image_size_stats": {
            "width": {
                "mean": df["image_width"].mean(),
                "std": df["image_width"].std(),
                "min": df["image_width"].min(),
                "max": df["image_width"].max(),
            },
            "height": {
                "mean": df["image_height"].mean(),
                "std": df["image_height"].std(),
                "min": df["image_height"].min(),
                "max": df["image_height"].max(),
            },
        },
        "bbox_size_stats": {
            "area_percent": {
                "mean": df[df["object_id"].notna()]["bbox_area_percent"].mean(),
                "std": df[df["object_id"].notna()]["bbox_area_percent"].std(),
                "min": df[df["object_id"].notna()]["bbox_area_percent"].min(),
                "max": df[df["object_id"].notna()]["bbox_area_percent"].max(),
            }
        },
    }

    return summary

In [ ]:
import hashlib
from pathlib import Path
from typing import Union, Optional

import pandas as pd


def _file_md5(path: Union[str, Path], chunk_size: int = 1024 * 1024) -> Optional[str]:
    """
    Возвращает MD5-хеш файла по его пути. Чтение идёт блоками, чтобы не загружать
    весь файл в память. В случае ошибки возвращает None.
    """
    try:
        md5 = hashlib.md5()
        with open(path, "rb") as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                md5.update(chunk)
        return md5.hexdigest()
    except Exception:
        return None


def extract_dataset_to_parquet(
    datasets_path: Union[str, Path],
    output_path: Union[str, Path],
    sample_size: Optional[int] = None,
    include_image_stats: bool = True,
) -> str:
    """
    Извлекает полную информацию о датасете и сохраняет в parquet файл

    Ищет изображения и аннотации только в следующих папках:
    - 01_train-s1__DataSet_Human_Rescue (сохраняется как train_s1)
    - 02_second_part_DataSet_Human_Rescue (сохраняется как train_s2)
    - 03_validation__DataSet_Human_Rescue (сохраняется как validation)

    Дополнительно для каждого изображения рассчитывается MD5-хеш файла (колонка `md5`).

    Args:
        datasets_path: Путь к папке содержащей папки датасетов (например '/mnt/data/dataset')
        output_path: Путь для сохранения parquet файла
        sample_size: Ограничить количество изображений для анализа (None = все)
        include_image_stats: Включать ли статистику изображений (размер файла и т.д.)

    Returns:
        Путь к созданному parquet файлу
    """

    datasets_path = Path(datasets_path)
    output_path = Path(output_path)

    if not datasets_path.exists():
        raise ValueError(f"Путь к датасетам не существует: {datasets_path}")

    output_path.parent.mkdir(parents=True, exist_ok=True)

    print(f"🔍 Сканирование датасетов в {datasets_path}")

    # Список для хранения всех данных
    all_data = []

    # Поддерживаемые форматы изображений
    image_extensions = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

    # Определенные папки для поиска
    target_datasets = [
        "01_train-s1__DataSet_Human_Rescue",
        "02_second_part_DataSet_Human_Rescue",
        "03_validation__DataSet_Human_Rescue",
        "04_ladd",
        "05_pd",
        "06_dataset_yolo9",
    ]

    # Находим изображения только в указанных папках датасетов
    all_images = []
    for dataset_name in target_datasets:
        dataset_path = datasets_path / dataset_name
        if dataset_path.exists():
            print(f"📁 Сканирую {dataset_name}...")
            for ext in image_extensions:
                found_images = list(dataset_path.rglob(f"*{ext}"))
                all_images.extend(found_images)
                if found_images:
                    print(f"   Найдено {len(found_images)} файлов {ext}")
        else:
            print(f"⚠️ Папка {dataset_name} не найдена")

    if sample_size and len(all_images) > sample_size:
        all_images = np.random.choice(all_images, sample_size, replace=False).tolist()

    print(f"📊 Найдено {len(all_images)} изображений для анализа")

    # Создаем словарь для быстрого поиска аннотаций в тех же папках
    print("🔍 Поиск аннотаций...")
    all_labels = []
    for dataset_name in target_datasets:
        dataset_path = datasets_path / dataset_name
        if dataset_path.exists():
            labels_in_dataset = list(dataset_path.rglob("*.txt"))
            all_labels.extend(labels_in_dataset)

    labels_dict = {label.stem: label for label in all_labels}

    print(f"📝 Найдено {len(all_labels)} файлов аннотаций")

    # Обрабатываем каждое изображение
    for img_path in tqdm(all_images, desc="Обработка изображений"):
        try:
            # MD5-хеш файла изображения
            md5_hash = _file_md5(img_path)

            # Базовая информация об изображении
            img_info = {
                "image_path": str(img_path),
                "image_name": img_path.name,
                "image_stem": img_path.stem,
                "dataset_name": _get_dataset_name(img_path, datasets_path),
                "relative_path": str(img_path.relative_to(datasets_path)),
                "md5": md5_hash,
            }

            # Загружаем изображение для получения размеров
            try:
                with Image.open(img_path) as img:
                    img_width, img_height = img.size
                    img_format = img.format
                    img_mode = img.mode

                img_info.update(
                    {
                        "image_width": img_width,
                        "image_height": img_height,
                        "image_format": img_format,
                        "image_mode": img_mode,
                        "image_aspect_ratio": img_width / img_height
                        if img_height
                        else None,
                        "image_total_pixels": img_width * img_height,
                    }
                )

                if include_image_stats:
                    size_bytes = img_path.stat().st_size
                    img_info.update(
                        {
                            "image_size_bytes": size_bytes,
                            "image_size_mb": size_bytes / (1024 * 1024),
                        }
                    )

            except Exception as e:
                print(f"⚠️ Ошибка при загрузке {img_path}: {e}")
                continue

            # Ищем соответствующую аннотацию
            annotation_path = labels_dict.get(img_path.stem)

            if annotation_path and annotation_path.exists():
                img_info["annotation_path"] = str(annotation_path)
                img_info["has_annotation"] = True

                # Парсим аннотации
                try:
                    with open(annotation_path, "r", encoding="utf-8") as f:
                        lines = [line.strip() for line in f.readlines() if line.strip()]

                    img_info["objects_count"] = len(lines)

                    if len(lines) > 0:
                        # Обрабатываем каждый объект
                        for obj_idx, line in enumerate(lines):
                            parts = line.split()
                            if len(parts) >= 5:
                                class_id = int(parts[0])
                                x_center = float(parts[1])
                                y_center = float(parts[2])
                                width_norm = float(parts[3])
                                height_norm = float(parts[4])

                                # Вычисляем координаты в пикселях
                                width_pix = width_norm * img_width
                                height_pix = height_norm * img_height
                                x_center_pix = x_center * img_width
                                y_center_pix = y_center * img_height

                                # Координаты углов
                                x1 = x_center_pix - width_pix / 2
                                y1 = y_center_pix - height_pix / 2
                                x2 = x_center_pix + width_pix / 2
                                y2 = y_center_pix + height_pix / 2

                                # Площадь и статистики
                                area_pix = width_pix * height_pix
                                area_percent = (
                                    area_pix / (img_width * img_height)
                                ) * 100
                                bbox_aspect_ratio = (
                                    width_pix / height_pix if height_pix > 0 else 0
                                )

                                # Создаем запись для каждого объекта
                                obj_data = img_info.copy()
                                obj_data.update(
                                    {
                                        "object_id": obj_idx,
                                        "class_id": class_id,
                                        "x_center_norm": x_center,
                                        "y_center_norm": y_center,
                                        "width_norm": width_norm,
                                        "height_norm": height_norm,
                                        "x_center_pix": x_center_pix,
                                        "y_center_pix": y_center_pix,
                                        "width_pix": width_pix,
                                        "height_pix": height_pix,
                                        "x1": x1,
                                        "y1": y1,
                                        "x2": x2,
                                        "y2": y2,
                                        "bbox_area_pix": area_pix,
                                        "bbox_area_percent": area_percent,
                                        "bbox_aspect_ratio": bbox_aspect_ratio,
                                    }
                                )

                                all_data.append(obj_data)
                    else:
                        # Изображение без объектов
                        img_info["objects_count"] = 0
                        img_info["object_id"] = None
                        all_data.append(img_info)

                except Exception as e:
                    print(f"⚠️ Ошибка при парсинге аннотации {annotation_path}: {e}")
                    img_info["objects_count"] = 0
                    img_info["annotation_error"] = str(e)
                    all_data.append(img_info)
            else:
                # Изображение без аннотации
                img_info["annotation_path"] = None
                img_info["has_annotation"] = False
                img_info["objects_count"] = 0
                img_info["object_id"] = None
                all_data.append(img_info)

        except Exception as e:
            print(f"⚠️ Общая ошибка при обработке {img_path}: {e}")
            continue

    # Создаем DataFrame
    print("📊 Создание DataFrame...")
    df = pd.DataFrame(all_data)

    # Сортируем по пути к изображению и id объекта
    df = df.sort_values(["image_path", "object_id"]).reset_index(drop=True)

    # Сохраняем в parquet
    parquet_path = output_path
    if not str(parquet_path).endswith(".parquet"):
        parquet_path = output_path / "dataset_info.parquet"

    print(f"💾 Сохранение в {parquet_path}")
    df.to_parquet(parquet_path, index=False)

    # Создаем summary
    summary = {
        "total_images": len(df["image_path"].unique()),
        "total_objects": int(df["object_id"].notna().sum()),
        "datasets": list(df["dataset_name"].unique()),
        "image_formats": list(df["image_format"].unique()),
        "has_annotations": int(df["has_annotation"].sum()),
        "images_without_annotations": int((~df["has_annotation"]).sum()),
        "columns": list(df.columns),
        "file_size_mb": parquet_path.stat().st_size / (1024 * 1024),
    }

    # Сохраняем summary
    summary_path = parquet_path.parent / f"{parquet_path.stem}_summary.json"
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)

    print("\n✅ Готово!")
    print(f"📁 Parquet файл: {parquet_path}")
    print(f"📋 Summary: {summary_path}")
    print(
        f"📊 Обработано: {summary['total_images']} изображений, {summary['total_objects']} объектов"
    )
    print(f"💾 Размер файла: {summary['file_size_mb']:.2f} MB")

    return str(parquet_path)


def _get_dataset_name(img_path: Path, base_path: Path) -> str:
    """Определяет название датасета по пути файла"""
    try:
        relative_path = img_path.relative_to(base_path)
        # Берем первую папку в относительном пути
        parts = relative_path.parts
        if len(parts) > 0:
            dataset_folder = parts[0]
            # Сокращаем длинные названия для удобства
            if dataset_folder == "01_train-s1__DataSet_Human_Rescue":
                return "train_s1"
            elif dataset_folder == "02_second_part_DataSet_Human_Rescue":
                return "train_s2"
            elif dataset_folder == "03_validation__DataSet_Human_Rescue":
                return "validation"
            else:
                return dataset_folder
        else:
            return "unknown"
    except Exception:
        return "unknown"


def load_dataset_info(parquet_path: str) -> pd.DataFrame:
    """
    Загружает информацию о датасете из parquet файла

    Args:
        parquet_path: Путь к parquet файлу

    Returns:
        DataFrame с информацией о датасете
    """
    return pd.read_parquet(parquet_path)


def get_dataset_summary(df: pd.DataFrame) -> dict:
    """
    Создает краткую сводку по датасету

    Args:
        df: DataFrame с информацией о датасете

    Returns:
        Словарь со статистикой
    """
    summary = {
        "total_images": len(df["image_path"].unique()),
        "total_objects": int(df["object_id"].notna().sum()),
        "avg_objects_per_image": df.groupby("image_path")["object_id"].count().mean(),
        "datasets": df["dataset_name"].value_counts().to_dict(),
        "image_size_stats": {
            "width": {
                "mean": df["image_width"].mean(),
                "std": df["image_width"].std(),
                "min": df["image_width"].min(),
                "max": df["image_width"].max(),
            },
            "height": {
                "mean": df["image_height"].mean(),
                "std": df["image_height"].std(),
                "min": df["image_height"].min(),
                "max": df["image_height"].max(),
            },
        },
        "bbox_size_stats": {
            "area_percent": {
                "mean": df[df["object_id"].notna()]["bbox_area_percent"].mean(),
                "std": df[df["object_id"].notna()]["bbox_area_percent"].std(),
                "min": df[df["object_id"].notna()]["bbox_area_percent"].min(),
                "max": df[df["object_id"].notna()]["bbox_area_percent"].max(),
            }
        },
    }

    return summary

In [ ]:
# Создание parquet файла
parquet_file = extract_dataset_to_parquet(
    datasets_path="./dataset",  # Ваш путь к датасетам
    output_path="./dataset_analysis.parquet",
    sample_size=None,  # None = все изображения
    include_image_stats=True,
)

# # Быстрая загрузка для анализа
# df = load_dataset_info("./dataset_analysis.parquet")
# summary = get_dataset_summary(df)

# # Анализ без картинок!
# outliers = df.nlargest(10, 'bbox_area_percent')
# small_objects = df[df['bbox_area_percent'] < 0.1]

In [ ]:
# 5% от каждого датасета
df = pl.read_parquet("dataset_analysis.parquet")
# sampled = df.group_by('dataset_name').map_groups(lambda x: x.sample(fraction=0.05, seed=42))
# sampled.write_csv('sample_dataset.csv')

In [ ]:
# sampled

In [ ]:
# Проверяем, есть ли дубликаты имён колонок
dupe_cols = [c for c in df.columns if df.columns.count(c) > 1]
print(
    f"Duplicate column names: {dupe_cols}"
    if dupe_cols
    else "All column names unique ✅"
)

# Процент пропусков «из коробки»
nulls = (
    df.null_count()  # <-- Series {col: n_nulls}
    .melt(variable_name="column", value_name="null_cnt")
    .with_columns((pl.col("null_cnt") / df.height * 100).round(2).alias("null_pct"))
    .filter(pl.col("null_cnt") > 0)
    .sort("null_cnt", descending=True)
)

nulls.head(20)

In [ ]:
df.columns

In [ ]:
import pandas as pd

# Загружаем данные
df = pd.read_parquet("dataset_analysis.parquet")

# Убираем дубликаты по image_path
df_img = (
    df[["image_path", "md5", "dataset_name"]]
    .drop_duplicates(subset=["image_path"])
    .dropna(subset=["md5"])
)

counts = df_img["md5"].value_counts()

# Кол-во групп дубликатов
num_duplicate_hashes = int((counts > 1).sum())

# Кол-во изображений в группах дубликатов
images_in_duplicate_hashes = int(counts[counts > 1].sum())

# Лишние файлы (в каждой группе один оригинал)
redundant_images = int((counts[counts > 1] - 1).sum())

print("Групп дубликатов (md5 с >1 файлом):", num_duplicate_hashes)
print("Изображений в группах дубликатов:", images_in_duplicate_hashes)
print("Лишних файлов (повторов сверх 1 на группу):", redundant_images)

# Таблица групп дубликатов с датасетами
dup_groups = (
    df_img.groupby("md5")
    .agg(
        count=("image_path", "count"),
        paths=("image_path", list),
        datasets=(
            "dataset_name",
            lambda x: sorted((x)),
        ),  # собираем уникальные датасеты
    )
    .reset_index()
    .query("count > 1")
    .sort_values("count", ascending=False)
)

print(dup_groups.head(10))

In [ ]:
dup_groups = (
    df_img.groupby("md5")
    .agg(
        count=("image_path", "count"),
        paths=("image_path", list),
        datasets=("dataset_name", lambda x: dict(pd.Series(x).value_counts())),
    )
    .reset_index()
    .query("count > 1")
    .sort_values("count", ascending=False)
)

print(dup_groups.head(10))

In [ ]:
from itertools import combinations
from collections import Counter

pair_counts = Counter()

for ds_dict in dup_groups["datasets"]:
    datasets = list(ds_dict.keys())
    if len(datasets) > 1:
        for combo in combinations(sorted(datasets), 2):
            pair_counts[combo] += 1

pair_df = pd.DataFrame(
    [(a, b, c) for (a, b), c in pair_counts.items()],
    columns=["dataset_1", "dataset_2", "duplicates"],
).sort_values("duplicates", ascending=False)

print(pair_df.head(10))

In [ ]:
# ===== Параметры =====
parquet_path = "dataset_analysis.parquet"  # <-- ваш parquet
export_dir: str | None = None  # например "md5_reports" или None
top_conflicts_preview = 20

# ===== 1) Уникальные изображения и md5-группы > 1 =====
df_img_lz = (
    pl.scan_parquet(parquet_path)
    .select(["image_path", "md5"])
    .unique(subset=["image_path"])  # по одному ряду на файл
    .filter(pl.col("md5").is_not_null() & (pl.col("md5") != ""))
)

md5_counts = df_img_lz.group_by("md5").agg(pl.len().alias("count"))

dup_md5_df = (
    md5_counts.filter(pl.col("count") > 1).sort("count", descending=True).collect()
)
dup_md5_list = dup_md5_df["md5"].to_list()

num_duplicate_hashes = int(dup_md5_df.height)
images_in_duplicate_hashes = int(dup_md5_df["count"].sum())
redundant_images = int((dup_md5_df["count"] - 1).sum())

print("==== MD5-дубликаты изображений ====")
print(f"MD5-групп с >1 файлом:              {num_duplicate_hashes}")
print(f"Изображений в группах дубликатов:   {images_in_duplicate_hashes}")
print(f"Лишних файлов (сверх 1 на группу):  {redundant_images}")

# ===== 2) Сигнатуры аннотаций по каждому изображению =====
df_rows_lz = (
    pl.scan_parquet(parquet_path)
    .select(
        [
            "image_path",
            "md5",
            "object_id",
            "class_id",
            "x_center_norm",
            "y_center_norm",
            "width_norm",
            "height_norm",
        ]
    )
    .filter(pl.col("md5").is_in(dup_md5_list))
)

# Формируем строку одной аннотации: "class x y w h" (округление до 6 знаков)
df_rows_lz = df_rows_lz.with_columns(
    [
        pl.when(pl.col("object_id").is_not_null())
        .then(
            pl.concat_str(
                [
                    pl.col("class_id").cast(pl.Int64, strict=False).cast(pl.Utf8),
                    pl.col("x_center_norm").round(6).cast(pl.Utf8),
                    pl.col("y_center_norm").round(6).cast(pl.Utf8),
                    pl.col("width_norm").round(6).cast(pl.Utf8),
                    pl.col("height_norm").round(6).cast(pl.Utf8),
                ],
                separator=" ",
            )
        )
        .otherwise(pl.lit(None))
        .alias("ann_row")
    ]
)

# Сбор строк аннотаций в список на изображение через .implode()
ann_per_image = (
    df_rows_lz.group_by(["image_path", "md5"])
    .agg(
        [
            pl.col("ann_row").drop_nulls().implode().alias("ann_rows"),  # -> List[str]
        ]
    )
    # Шаг 1: производные колонки из ann_rows (НЕ ссылаемся на новые алиасы в этом же вызове)
    .with_columns(
        [
            pl.col("ann_rows").list.sort().alias("ann_rows_sorted"),
            pl.col("ann_rows").list.len().alias("objects_count"),
        ]
    )
    # Шаг 2: теперь можно использовать созданные aliase'ы
    .with_columns(
        [
            (pl.col("objects_count") > 0).alias("has_ann"),
            pl.when(pl.col("objects_count") > 0)
            .then(pl.col("ann_rows_sorted").list.join("\n"))
            .otherwise(pl.lit(""))
            .alias("ann_text"),
        ]
    )
    # Хэш содержимого аннотаций (или "NOANN", если пусто)
    .with_columns(
        [
            pl.col("ann_text")
            .map_elements(
                lambda s: hashlib.md5(s.encode("utf-8")).hexdigest() if s else "NOANN"
            )
            .alias("ann_sig")
        ]
    )
    .select(["image_path", "md5", "ann_sig", "has_ann", "objects_count"])
    .collect()
)

# ===== 3) Классификация md5-групп по статусу аннотаций =====
# Разбивка md5 × ann_sig -> count, paths
per_md5_sig = (
    ann_per_image.lazy()
    .group_by(["md5", "ann_sig"])
    .agg(
        [
            pl.len().alias("count"),
            pl.col("image_path").implode().alias("paths"),
        ]
    )
)

# md5 -> group_size, число вариантов, есть ли NOANN, списки вариантов/размеров
per_md5 = (
    per_md5_sig.group_by("md5")
    .agg(
        [
            pl.sum("count").alias("group_size"),
            pl.len().alias("num_annotation_variants"),
            (pl.col("ann_sig") == "NOANN").any().alias("has_noann"),
            pl.col("ann_sig").implode().alias("ann_sigs"),
            pl.col("count").implode().alias("ann_sig_counts"),
        ]
    )
    # variants_excluding_noann = num_annotation_variants - (has_noann ? 1 : 0)
    .with_columns(
        [
            pl.when(pl.col("has_noann"))
            .then(pl.col("num_annotation_variants") - 1)
            .otherwise(pl.col("num_annotation_variants"))
            .alias("variants_excluding_noann")
        ]
    )
    # финальный статус
    .with_columns(
        [
            pl.when((pl.col("num_annotation_variants") == 1) & pl.col("has_noann"))
            .then(pl.lit("all_missing"))
            .when((pl.col("variants_excluding_noann") == 1) & (~pl.col("has_noann")))
            .then(pl.lit("all_identical"))
            .when(pl.col("has_noann") & (pl.col("variants_excluding_noann") >= 1))
            .then(pl.lit("partial_missing"))
            .otherwise(pl.lit("conflict"))
            .alias("ann_status")
        ]
    )
    .collect()
)

# Оставим только реальные группы (>1 файла)
per_md5 = per_md5.filter(pl.col("group_size") > 1)

# Метрики по статусам
n_groups = per_md5.height
n_all_identical = per_md5.filter(pl.col("ann_status") == "all_identical").height
n_all_missing = per_md5.filter(pl.col("ann_status") == "all_missing").height
n_partial_missing = per_md5.filter(pl.col("ann_status") == "partial_missing").height
n_conflict = per_md5.filter(pl.col("ann_status") == "conflict").height

print("\n==== Аннотации по md5-группам ====")
print(f"Всего md5-групп (>1 файл):          {n_groups}")
print(f"  одинаковые аннотации:             {n_all_identical}")
print(f"  у всех отсутствуют:               {n_all_missing}")
print(f"  частично отсутствуют:             {n_partial_missing}")
print(f"  КОНФЛИКТ (разные аннотации):      {n_conflict}")

# Развёртка конфликтов (md5 × ann_sig)
per_md5_sig_df = per_md5_sig.collect()
conflict_md5 = per_md5.filter(pl.col("ann_status") == "conflict").select(
    ["md5", "group_size"]
)
conflict_breakdown = per_md5_sig_df.join(conflict_md5, on="md5", how="inner").sort(
    ["group_size", "count"], descending=[True, True]
)

print("\n==== Пример конфликтных групп (первые строки) ====")
print(conflict_breakdown.head(top_conflicts_preview))

# ===== 4) Дублирующиеся боксы внутри одного изображения =====
dup_boxes = (
    df_rows_lz.filter(pl.col("ann_row").is_not_null())
    .group_by(["image_path", "ann_row"])
    .agg(pl.len().alias("count"))
    .filter(pl.col("count") > 1)
    .collect()
)

print("\n==== Дублирующиеся боксы внутри изображения (первые строки) ====")
print(dup_boxes.head(20))

# ===== 5) (опционально) Сохранить результаты =====
if export_dir is not None:
    out = Path(export_dir)
    out.mkdir(parents=True, exist_ok=True)
    dup_md5_df.write_csv(out / "md5_duplicate_groups_counts.csv")
    per_md5.write_csv(out / "md5_annotation_status.csv")
    conflict_breakdown.write_csv(out / "md5_annotation_conflicts.csv")
    dup_boxes.write_csv(out / "duplicate_boxes_within_image.csv")
    print(f"\nФайлы сохранены в: {out.resolve()}")

# На выходе доступны:
# dup_md5_df         -> md5 и количество файлов (только группы >1)
# ann_per_image      -> по 1 ряду на изображение: ann_sig / has_ann / objects_count
# per_md5            -> сводка по md5-группам и их статус
# conflict_breakdown -> разрез по конфликтным md5: (md5, ann_sig, count, paths, group_size)
# dup_boxes          -> повторяющиеся строки YOLO внутри одного image_path

In [ ]:
obj_dist = (
    df.group_by("image_stem")
    .agg(pl.max("objects_count").alias("objects"))
    .group_by("objects")
    .count()
    .sort("objects")
)
print(obj_dist)

In [ ]:
# 1) Кадр → people count
obj_per_frame = df.group_by("image_stem").agg(pl.max("objects_count").alias("objects"))

# 2) Распределение
obj_dist = (obj_per_frame.group_by("objects").count().sort("objects")).to_pandas()

# 3) превращаем X‑ось в категорию
obj_dist["objects"] = obj_dist["objects"].astype(str)

fig = px.bar(
    obj_dist,
    x="objects",
    y="count",
    title="Распределение objects_count (уникальные кадры)",
    labels={"objects": "objects_count", "count": "кадров"},
    text_auto=True,  # подписи над барами (опц.)
)
fig.update_layout(
    xaxis_title="objects_count", yaxis_title="кадров", bargap=0.1
)  # чуть уже зазоры
fig.show()

In [ ]:
# Предположим, столбцы image_width / image_height уже посчитаны.
# Если их нет, можно прочитать размеры прямо из файлов (медленнее).
size_stats = (
    df.group_by("image_stem")
    .first()
    .select("image_stem", "image_width", "image_height")
    .unique()
    .select(
        [
            pl.mean("image_width").alias("avg_w"),
            pl.mean("image_height").alias("avg_h"),
            pl.min("image_width").alias("min_w"),
            pl.max("image_width").alias("max_w"),
            pl.min("image_height").alias("min_h"),
            pl.max("image_height").alias("max_h"),
        ]
    )
)
print(size_stats)

# Соотношение сторон
ratio_df = df.select(
    "image_stem", (pl.col("image_width") / pl.col("image_height")).alias("aspect")
).unique()

px.histogram(
    ratio_df.to_pandas(), x="aspect", nbins=40, title="Соотношение сторон кадров"
).show()

# Экстремальные файлы
tiny = df.filter((pl.col("image_width") < 320) | (pl.col("image_height") < 320))
huge = df.filter((pl.col("image_width") > 4000) | (pl.col("image_height") > 4000))
print(f"Tiny frames: {tiny.select('image_stem').n_unique()}")
print(f"Huge frames: {huge.select('image_stem').n_unique()}")

In [ ]:
bbox_x, bbox_y = "x1", "y1"  # левый‑верхний угол
bbox_w, bbox_h = "width_pix", "height_pix"
img_w, img_h = "image_width", "image_height"

bbox = df.filter(pl.col("objects_count") > 0).with_columns(
    [
        (pl.col(bbox_w) * pl.col(bbox_h)).alias("bbox_area_pix"),
        (
            (pl.col(bbox_w) * pl.col(bbox_h)) / (pl.col(img_w) * pl.col(img_h)) * 100
        ).alias("bbox_pct"),
        (pl.col(bbox_w) / pl.col(bbox_h)).alias("bbox_aspect"),
    ]
)

px.histogram(
    bbox.to_pandas(), x="bbox_pct", nbins=50, title="BBox area (% кадра)"
).show()
px.histogram(
    bbox.to_pandas(), x="bbox_aspect", nbins=50, title="BBox aspect ratio"
).show()

In [ ]:
# Jupyter cell 6 ────────────────────────────────────────────────────────────────
import polars as pl
import matplotlib.pyplot as plt


def safe_sample(df_sub, n=5, seed=42):
    k = min(df_sub.height, n)
    return df_sub.sample(k, seed=seed) if k > 0 else pl.DataFrame()


suspect_big = bbox.filter(pl.col("bbox_pct") > 70)
suspect_small = bbox.filter(pl.col("bbox_pct") < 1)

print("Big bbox frames:", suspect_big.select("image_stem").n_unique())
print("Small bbox frames:", suspect_small.select("image_stem").n_unique())

examples_df = pl.concat([safe_sample(suspect_big), safe_sample(suspect_small)])

if examples_df.is_empty():
    print("⚠️  Нет примеров, удовлетворяющих критериям.")
else:
    thumbs = []
    for rec in examples_df.iter_rows(named=True):
        img = Image.open(rec["image_path"]).convert("RGB")

        # коэффициент ресайза относительно оригинального кадра
        scale = 640 / max(rec["image_width"], rec["image_height"])
        if scale < 1.0:  # только уменьшаем
            new_size = (
                int(rec["image_width"] * scale),
                int(rec["image_height"] * scale),
            )
            img = img.resize(new_size, Image.LANCZOS)

        # пересчёт bbox
        x = rec[bbox_x] * scale
        y = rec[bbox_y] * scale
        w = rec[bbox_w] * scale
        h = rec[bbox_h] * scale

        draw = ImageDraw.Draw(img)
        draw.rectangle([x, y, x + w, y + h], outline="red", width=6)
        thumbs.append(img)

    # --- вывод сеткой ---
    cols = 3
    rows = int(np.ceil(len(thumbs) / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))

    for ax, im in zip(np.asarray(axes).ravel(), thumbs):
        ax.imshow(im)
        ax.axis("off")

    for ax in np.asarray(axes).ravel()[len(thumbs) :]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
TARGET_IMG = "example_frame_00123"  # ← поставьте нужный image_stem

boxes = df.filter(pl.col("image_stem") == TARGET_IMG)

if boxes.is_empty():
    print("🙁 Такой картинки нет.")
else:
    img_path = boxes.select("image_path").item(0)
    img = Image.open(img_path).convert("RGB")
    drw = ImageDraw.Draw(img)

    for row in boxes.iter_rows(named=True):
        drw.rectangle(
            [
                row[bbox_x],
                row[bbox_y],
                row[bbox_x] + row[bbox_w],
                row[bbox_y] + row[bbox_h],
            ],
            outline="orange",
            width=3,
        )

    display(img)

In [ ]:
# Jupyter cell 8 ────────────────────────────────────────────────────────────────
heat = df.select(["x_center_norm", "y_center_norm"])
fig = px.density_heatmap(
    heat.to_pandas(),
    x="x_center_norm",
    y="y_center_norm",
    nbinsx=50,
    nbinsy=50,
    title="Тепловая карта центров bbox (нормированные координаты)",
)
fig.update_yaxes(autorange="reversed")  # чтобы (0,0) было в левом‑верхнем
fig.show()

In [ ]:
# Jupyter cell 9 ────────────────────────────────────────────────────────────────
corr = df.group_by("image_stem").agg(
    [
        pl.mean("bbox_area_percent").alias("avg_bbox_pct"),
        pl.max("objects_count").alias("objects"),
    ]
)

fig = px.scatter(
    corr.to_pandas(),
    x="objects",
    y="avg_bbox_pct",
    trendline="ols",
    title="Средняя площадь bbox (%) vs число людей",
)
fig.update_traces(marker=dict(size=6))
fig.show()

In [ ]:
# Jupyter cell 10 ──────────────────────────────────────────────────────────────
dup_boxes = (
    df.filter(pl.col("objects_count") > 1)
    .group_by(["image_stem", "x1", "y1", "x2", "y2"])
    .count()
    .filter(pl.col("count") > 1)
)

print("Полных дублей bbox:", dup_boxes.shape[0])
dup_boxes.head(10)

In [ ]:
# 1) Если нет bbox_area_percent, пересчитываем
if "bbox_area_percent" not in df.columns:
    df = df.with_columns(
        (
            (pl.col("width_pix") * pl.col("height_pix"))
            / (pl.col("image_width") * pl.col("image_height"))
            * 100
        ).alias("bbox_area_percent")
    )

# 2) Добавляем флаги только к строкам с объектами
df = df.with_columns(
    [
        pl.when(pl.col("objects_count") > 0)  # фильтр рамок
        .then((pl.col("bbox_area_percent") > 50) | (pl.col("bbox_area_percent") < 0.25))
        .otherwise(False)
        .alias("bbox_outlier"),
        pl.when(pl.col("objects_count") > 0)
        .then(
            (pl.col("x1") < 0)
            | (pl.col("y1") < 0)
            | (pl.col("x2") > pl.col("image_width"))
            | (pl.col("y2") > pl.col("image_height"))
        )
        .otherwise(False)
        .alias("bbox_oob"),
    ]
)

# 3) Быстрый просмотр, сколько нашли
summary = df.select(
    [
        pl.col("bbox_outlier").sum().alias("outliers"),
        pl.col("bbox_oob").sum().alias("out_of_bounds"),
    ]
)
print(summary)

In [ ]:
dirty_frames = (
    df.filter(pl.col("bbox_outlier") | pl.col("bbox_oob")).select("image_stem").unique()
)

print(f"Найдено {dirty_frames.height} кадров с потенциальными проблемами.")

In [ ]:
q01 = (
    df.filter(pl.col("objects_count") > 0)
    .select(pl.col("bbox_area_percent").quantile(0.01))
    .item(0, 0)  # <─ row=0, col=0
)

# 2) преобразовать в Series и взять 1‑й элемент
q99 = (
    df.filter(pl.col("objects_count") > 0)
    .select(pl.col("bbox_area_percent").quantile(0.99))
    .to_series()[0]  # <─ Series → скаляр
)

print(f"1‑й процентиль:  {q01:.4f} %")
print(f"99‑й процентиль: {q99:.4f} %")

In [ ]:
# Jupyter cell — пересчёт флагов по новым порогам
TINY_PCT = 0.01  # 0.01 %
HUGE_PCT = 1.0  # 1.0 %

df = df.with_columns(
    [
        # tiny/huge только для строк с bbox
        pl.when(pl.col("objects_count") > 0)
        .then(
            (pl.col("bbox_area_percent") < TINY_PCT)
            | (pl.col("bbox_area_percent") > HUGE_PCT)
        )
        .otherwise(False)
        .alias("bbox_outlier"),
        pl.when(pl.col("objects_count") > 0)
        .then(
            (pl.col("x1") < 0)
            | (pl.col("y1") < 0)
            | (pl.col("x2") > pl.col("image_width"))
            | (pl.col("y2") > pl.col("image_height"))
        )
        .otherwise(False)
        .alias("bbox_oob"),
    ]
)

# Сводка
summary = df.select(
    [
        pl.col("bbox_outlier").sum().alias("outliers"),
        pl.col("bbox_oob").sum().alias("out_of_bounds"),
    ]
)
print(summary)

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output


# ====== параметры ======
ID_COL = "image_stem"  # ← можно сменить на "image_name" и т.п.
page_size = 10  # по сколько кадров на страницу
max_side = 640  # макс. размер длинной стороны миниатюры

# ---------- подготовка списка проблемных кадров ----------
flagged_df = (
    df.filter(pl.col("bbox_outlier") | pl.col("bbox_oob")).select(ID_COL).unique()
)
flagged_list = flagged_df.to_series().to_list()
total_frames = len(flagged_list)

# словарь: id -> порядковый номер (для подписи №)
idx_map = {idv: i for i, idv in enumerate(flagged_list)}


def draw_page(start_idx):
    clear_output(wait=True)
    subset = flagged_list[start_idx : start_idx + page_size]
    if not subset:
        print("📭  Кадров больше нет.")
        return start_idx

    # Подтягиваем строки по кадрам
    view_df = df.filter(pl.col(ID_COL).is_in(subset))

    thumbs = []
    labels = []
    for stem in subset:
        rows = view_df.filter(pl.col(ID_COL) == stem)
        rec0 = rows.row(0, named=True)
        img = Image.open(rec0["image_path"]).convert("RGB")

        # масштаб
        scale = max_side / max(rec0["image_width"], rec0["image_height"])
        if scale < 1.0:
            img = img.resize(
                (int(rec0["image_width"] * scale), int(rec0["image_height"] * scale)),
                Image.LANCZOS,
            )

        # рамки
        drw = ImageDraw.Draw(img)
        for r in rows.iter_rows(named=True):
            x1 = r["x1"] * scale
            y1 = r["y1"] * scale
            x2 = r["x2"] * scale
            y2 = r["y2"] * scale
            color = "red" if r["bbox_oob"] else (144, 238, 144)
            drw.rectangle([x1, y1, x2, y2], outline=color, width=7)

        thumbs.append(img)
        labels.append(f"#{idx_map[stem]} | {stem}")  # подпись

    # --- рисуем сетку ---
    cols = 5
    rows = int(np.ceil(len(thumbs) / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3.2, rows * 3.5))

    flat_axes = np.asarray(axes).ravel() if hasattr(axes, "ravel") else [axes]
    for ax, im, lab in zip(flat_axes, thumbs, labels):
        ax.imshow(im)
        ax.set_title(lab, fontsize=8)
        ax.axis("off")

    for ax in flat_axes[len(thumbs) :]:
        ax.axis("off")

    plt.suptitle(
        f"Flagged frames {start_idx}–{start_idx + len(thumbs) - 1} / {total_frames}",
        fontsize=14,
    )
    plt.tight_layout()
    plt.show()

    return start_idx


# ---------- виджеты ----------
idx = 0

prev_btn = widgets.Button(description="← Prev", layout=widgets.Layout(width="80px"))
next_btn = widgets.Button(description="Next →", layout=widgets.Layout(width="80px"))
out = widgets.Output()


def on_prev(_):
    global idx
    idx = max(idx - page_size, 0)
    with out:
        idx = draw_page(idx)


def on_next(_):
    global idx
    idx = min(idx + page_size, (total_frames - 1) // page_size * page_size)
    with out:
        idx = draw_page(idx)


prev_btn.on_click(on_prev)
next_btn.on_click(on_next)

display(widgets.HBox([prev_btn, next_btn]))
with out:
    idx = draw_page(idx)  # initial page
display(out)

In [ ]:
frame_stats = (
    df.group_by("image_stem")
    .agg(
        [
            pl.max("objects_count").alias("objects"),
            pl.first("image_total_pixels").alias("pixels"),
        ]
    )
    .with_columns((pl.col("objects") / (pl.col("pixels") / 1_000_000)).alias("density"))
)
px.histogram(
    frame_stats.to_pandas(),
    x="density",
    nbins=50,
    title="Плотность людей (объектов на 1 Мп)",
).show()

In [ ]:
!pip install ensemble-boxes ipywidgets